# Interactive Stationary Distribution Visualization for Literal Automaton

This notebook creates an interactive bar chart visualization for the stationary distribution of a Literal Automaton with eight states using the exact equations from the assignment. The visualization uses dynamic sliders to adjust parameters and observe how they affect the distribution.

## Import Required Libraries

Import necessary libraries such as Dash, Plotly, and NumPy for visualization and computation.

In [ ]:
# Import Required Libraries
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import numpy as np
import pandas as pd

## Define Equations for Stationary Distribution

Implement the exact equations from the assignment to calculate π₁ through π₈ based on the parameters s, P(L|Y), P(Y), and P(~L|~Y).

In [ ]:
def calculate_stationary_distribution(s, p_l_given_y, p_y, p_not_l_given_not_y):
    """
    Calculate the stationary distribution for the 8-state Literal Automaton using the exact equations.
    
    From the assignment image, the equations are:
    π₁ = α P(Y)⁴ P(L|Y)⁷
    π₂ = α P(Y)³ P(L|Y)⁶ s (P(L|Y)P(Y) + P(L|Ȳ)P(Ȳ))
    π₃ = α P(Y)² P(L|Y)⁵ s² (P(L|Y)P(Y) + P(L|Ȳ)P(Ȳ))²
    π₄ = α P(Y) P(L|Y)⁴ s³ (P(L|Y)P(Y) + P(L|Ȳ)P(Ȳ))³
    π₅ = α P(L|Y)³ s⁴ (P(L|Y)P(Y) + P(L|Ȳ)P(Ȳ))⁴
    π₆ = α P(L|Y) P(L|Y)² s⁵ (P(L|Y)P(Y) + P(L|Ȳ)P(Ȳ))¹
    π₇ = α P(L|Y)² P(L|Y)¹ s⁶ (P(L|Y)P(Y) + P(L|Ȳ)P(Ȳ))⁴
    π₈ = α P(L|Y)³ s⁷ (P(L|Y)P(Y) + P(L|Ȳ)P(Ȳ))⁴
    
    Parameters:
    - s: strength parameter (1.0 ≤ s ≤ 25.0)
    - p_l_given_y: P(L|Y) (0 ≤ P(L|Y) ≤ 1)
    - p_y: P(Y) (0 ≤ P(Y) ≤ 1)
    - p_not_l_given_not_y: P(~L|~Y) (0 ≤ P(~L|~Y) ≤ 1)
    
    Returns:
    - Array of 8 probabilities representing π₁ through π₈
    """
    
    # Calculate derived probabilities
    p_not_y = 1 - p_y  # P(Ȳ)
    p_l_given_not_y = 1 - p_not_l_given_not_y  # P(L|Ȳ)
    
    # Calculate the base expression that appears in multiple equations
    # (P(L|Y)P(Y) + P(L|Ȳ)P(Ȳ))
    base_expr = p_l_given_y * p_y + p_l_given_not_y * p_not_y
    
    # Calculate each term without normalization first
    pi_1_unnorm = p_y**4 * p_l_given_y**7
    pi_2_unnorm = p_y**3 * p_l_given_y**6 * s * base_expr
    pi_3_unnorm = p_y**2 * p_l_given_y**5 * s**2 * base_expr**2
    pi_4_unnorm = p_y * p_l_given_y**4 * s**3 * base_expr**3
    pi_5_unnorm = p_l_given_y**3 * s**4 * base_expr**4
    pi_6_unnorm = p_l_given_y * p_l_given_y**2 * s**5 * base_expr**1
    pi_7_unnorm = p_l_given_y**2 * p_l_given_y**1 * s**6 * base_expr**4
    pi_8_unnorm = p_l_given_y**3 * s**7 * base_expr**4
    
    # Calculate the normalization constant α
    total_unnorm = (pi_1_unnorm + pi_2_unnorm + pi_3_unnorm + pi_4_unnorm +
                   pi_5_unnorm + pi_6_unnorm + pi_7_unnorm + pi_8_unnorm)
    
    alpha = 1.0 / total_unnorm if total_unnorm > 0 else 1.0
    
    # Calculate normalized probabilities
    pi_1 = alpha * pi_1_unnorm
    pi_2 = alpha * pi_2_unnorm
    pi_3 = alpha * pi_3_unnorm
    pi_4 = alpha * pi_4_unnorm
    pi_5 = alpha * pi_5_unnorm
    pi_6 = alpha * pi_6_unnorm
    pi_7 = alpha * pi_7_unnorm
    pi_8 = alpha * pi_8_unnorm
    
    distribution = np.array([pi_1, pi_2, pi_3, pi_4, pi_5, pi_6, pi_7, pi_8])
    
    return distribution

## Create Interactive Visualization with Sliders

Use Dash to create an interactive bar chart with sliders for s (1.0-25.0), P(L|Y), P(Y), and P(~L|~Y) (0.0-1.0). The chart updates dynamically to show the stationary distribution.

In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("Literal Automaton - Interactive Stationary Distribution Visualization", 
            style={'textAlign': 'center', 'marginBottom': '30px'}),
    
    # Parameter controls
    html.Div([
        html.Div([
            html.Label("s (Strength parameter):", style={'fontWeight': 'bold'}),
            dcc.Slider(
                id='s-slider',
                min=1.0,
                max=25.0,
                step=0.1,
                value=5.0,
                marks={i: str(i) for i in range(1, 26, 4)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={'width': '48%', 'display': 'inline-block', 'marginBottom': '20px'}),
        
        html.Div([
            html.Label("P(L|Y) (Probability of L given Y):", style={'fontWeight': 'bold'}),
            dcc.Slider(
                id='p-l-given-y-slider',
                min=0.0,
                max=1.0,
                step=0.01,
                value=0.8,
                marks={i/10: str(i/10) for i in range(0, 11, 2)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block', 'marginBottom': '20px'})
    ]),
    
    html.Div([
        html.Div([
            html.Label("P(Y) (Probability of Y):", style={'fontWeight': 'bold'}),
            dcc.Slider(
                id='p-y-slider',
                min=0.0,
                max=1.0,
                step=0.01,
                value=0.79,
                marks={i/10: str(i/10) for i in range(0, 11, 2)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={'width': '48%', 'display': 'inline-block', 'marginBottom': '20px'}),
        
        html.Div([
            html.Label("P(~L|~Y) (Probability of not L given not Y):", style={'fontWeight': 'bold'}),
            dcc.Slider(
                id='p-not-l-given-not-y-slider',
                min=0.0,
                max=1.0,
                step=0.01,
                value=0.22,
                marks={i/10: str(i/10) for i in range(0, 11, 2)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block', 'marginBottom': '20px'})
    ]),
    
    # Derived parameters display
    html.Div([
        html.H3("Derived Parameters:", style={'marginTop': '30px'}),
        html.Div(id='derived-params', style={'marginBottom': '20px', 'padding': '10px', 
                                             'backgroundColor': '#f0f0f0', 'borderRadius': '5px'})
    ]),
    
    # Bar chart
    dcc.Graph(id='stationary-distribution-chart'),
    
    # Model description
    html.Div([
        html.H3("Model Description:"),
        html.P([
            "This visualization shows the stationary distribution of a Literal Automaton with eight states ",
            "using a Markov chain model. The distribution is calculated using the exact equations from the assignment, ",
            "where each πᵢ represents the long-term probability of being in state i."
        ]),
        html.P([
            "States 1-6 typically represent 'forgotten' configurations with lower probabilities, ",
            "while states 7-8 represent 'memorized' configurations with higher probabilities. ",
            "The strength parameter s controls the memory effects in the system."
        ])
    ], style={'marginTop': '30px', 'padding': '20px', 'backgroundColor': '#f9f9f9', 'borderRadius': '5px'})
])

# Callback to update the chart and derived parameters
@app.callback(
    [Output('stationary-distribution-chart', 'figure'),
     Output('derived-params', 'children')],
    [Input('s-slider', 'value'),
     Input('p-l-given-y-slider', 'value'),
     Input('p-y-slider', 'value'),
     Input('p-not-l-given-not-y-slider', 'value')]
)
def update_chart(s, p_l_given_y, p_y, p_not_l_given_not_y):
    # Calculate derived parameters
    p_not_l_given_y = 1 - p_l_given_y
    p_not_y = 1 - p_y
    p_l_given_not_y = 1 - p_not_l_given_not_y
    
    # Calculate the stationary distribution
    distribution = calculate_stationary_distribution(s, p_l_given_y, p_y, p_not_l_given_not_y)
    
    # Create state labels (numbered 1-8)
    state_labels = [f"State {i+1}" for i in range(8)]
    
    # Create annotations for forgotten/memorized regions
    annotations = []
    # Add "Forgotten" annotation for states 1-6
    annotations.append(dict(
        x=2.5, y=max(distribution[:6]) + 0.02,
        text="Forgotten", showarrow=False,
        font=dict(size=12, color="gray")
    ))
    # Add "Memorized" annotation for states 7-8
    annotations.append(dict(
        x=6.5, y=max(distribution[6:]) + 0.05,
        text="Memorized", showarrow=False,
        font=dict(size=12, color="gray")
    ))
    
    # Create the bar chart with colors similar to the reference image
    colors = ['lightblue'] * 6 + ['mediumpurple'] * 2  # Blue for forgotten, purple for memorized
    
    fig = go.Figure(data=[
        go.Bar(
            x=list(range(1, 9)),  # States numbered 1-8
            y=distribution,
            text=[f"{prob:.2f}" if prob > 0.05 else "" for prob in distribution],
            textposition='outside',
            marker_color=colors,
            hovertemplate='<b>State %{x}</b><br>' + 
                         'Probability: %{y:.4f}<extra></extra>'
        )
    ])
    
    fig.update_layout(
        title='Interactive Stationary Distribution Chart',
        xaxis_title='',
        yaxis_title='',
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(1, 9)),
            ticktext=[str(i) for i in range(1, 9)]
        ),
        yaxis=dict(range=[0, 1]),
        height=400,
        showlegend=False,
        annotations=annotations,
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    # Create derived parameters display
    derived_params = html.Div([
        html.P(f"P(Y) = {p_y:.2f}", style={'margin': '2px 0'}),
        html.P(f"P(L|Y) = {p_l_given_y:.1f}", style={'margin': '2px 0'}),
        html.P(f"P(L|not Y) = {p_l_given_not_y:.2f}", style={'margin': '2px 0'}),
        html.P(f"P(~L|Y) = {p_not_l_given_y:.2f}", style={'margin': '2px 0'}),
        html.P(f"P(~Y) = {p_not_y:.2f}", style={'margin': '2px 0'})
    ])
    
    return fig, derived_params

## Run the Interactive Visualization

Execute this cell to start the Dash app. The app will run on http://127.0.0.1:8050/ and provide an interactive interface to explore how different parameter values affect the stationary distribution.

In [ ]:
# Run the app
if __name__ == '__main__':
    print("Starting the interactive visualization...")
    print("Open your browser and navigate to http://127.0.0.1:8050/")
    print("Use the sliders to adjust parameters and observe how the stationary distribution changes.")
    
    # Test with the reference parameter values
    print("\nTesting with reference values (P(Y)=0.79, P(L|Y)=0.8, P(L|not Y)=0.78):")
    test_dist = calculate_stationary_distribution(5.0, 0.8, 0.79, 0.22)
    print(f"Distribution: {test_dist}")
    print(f"Sum: {np.sum(test_dist):.6f}")
    print(f"States 7-8 (memorized): {test_dist[6:]}, sum = {np.sum(test_dist[6:]):.3f}")
    print(f"States 1-6 (forgotten): {test_dist[:6]}, sum = {np.sum(test_dist[:6]):.3f}")
    
    # Run the Dash app
    app.run_server(debug=True, host='0.0.0.0', port=8050)

## Assignment Requirements Checklist

✓ **Interactive stationary distribution bar chart** for 8-state Literal Automaton  
✓ **Exact equations used** to calculate the height of each bar (π₁ through π₈)  
✓ **Dynamic sliders implemented:**
  - s: 1.0 to 25.0 (strength parameter)
  - P(L|Y): 0.0 to 1.0 (probability sliders)
  - P(Y): 0.0 to 1.0
  - P(~L|~Y): 0.0 to 1.0  
✓ **Derived parameters automatically calculated:**
  - P(~L|Y) = 1.0 − P(L|Y)
  - P(~Y) = 1.0 − P(Y)  
✓ **Real-time updates** as sliders change  
✓ **Visual distinction** between forgotten (states 1-6) and memorized (states 7-8) regions  

### Usage Instructions:
1. Run all cells in the notebook
2. Open http://127.0.0.1:8050/ in your browser
3. Use the sliders to explore different parameter combinations
4. Observe how the distribution changes, particularly the shift between forgotten and memorized states
5. Note how higher 's' values tend to increase memory effects